In [1]:
import os
import pathlib
import glob
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import gemmi

In [30]:
import subprocess

## Goal:    
Continous plot of values (B factor, occupancy, etc.) of water molecules by their distance (to the closest residue).

In [8]:
water_filter = ['HOH', 'WAT', 'H2O'] # List of water residue names.
AA=['ALA', 'ARG', 'ASN', 'ASP', 'CYS', 'GLN', 'GLU', 'GLY', 'HIS', 'ILE', 'LEU', 'LYS', 'MET', 'PHE', 'PRO', 'SER', 'THR', 'TRP', 'TYR', 'VAL'] # amino acids.


In [9]:
# Loop through all the PDB files in the data folder.
# Training set.
train_list = pd.read_csv("/Users/mingbin/Desktop/Wankowicz_lab_Mac/Projects/water/data/train_split.txt", header=None)
train_list = train_list[0].tolist()
# Test set.
test_list = pd.read_csv("/Users/mingbin/Desktop/Wankowicz_lab_Mac/Projects/water/data/test_split.txt", header=None)
test_list = test_list[0].tolist()

pdb_path = pathlib.Path('/Users/mingbin/Desktop/Wankowicz_lab_Mac/Projects/water/data/pdb_redo_data')
pdbs = list(pdb_path.glob('*_final.pdb')) # Pay attention to the pattern of names. A general "*.pdb" would result in duplicate files when looping.
# Train.
pdb_train = []
for a in pdbs:
    name = a.stem.split('_')[0]
    if name in train_list:
        pdb_train.append(a)
# Test.
pdb_test = []
for a in pdbs:
    name = a.stem.split('_')[0]
    if name in test_list:
        pdb_test.append(a)

*Instead of storing information of all atoms in a structure, calculate the distance on-the-fly, otherwise time takes too long.*

In [13]:
def calc_distance(coord1, coord2):
    """Calculate the distance between two sets of coordinates."""
    try:
        len(coord1), len(coord2)
    except:
        raise ValueError("The input must be a list of coordinates.")
    distance = np.sqrt(np.sum((np.array(coord1) - np.array(coord2))**2))
    return distance

In [14]:
# test for one single structure first.
structure = gemmi.read_structure("/Users/mingbin/Desktop/Wankowicz_lab_Mac/Projects/water/data/pdb_redo_data/2yp9_final.pdb")
# Check model.
i = 0
model = structure[i]  # consider the first model (skip if empty)
while len(model) == 0:  # sometimes the first model is empty
    i += 1
    try:
        model = structure[i]
    except Exception:
        raise ValueError("Can't read valid model from the input PDB file!")

wat_chain_dict, AA_chain_dict={}, {}
for chain in model:
    wat_chain_dict[chain.name] = []
    AA_chain_dict[chain.name] = []
    for res in chain:
        # Distinguish between water and amino acid.
        if res.name in water_filter: # water molecules.
            for atom in res: # Store a list of information for each atom.
                wat_chain_dict[chain.name].append([res.name, str(res.seqid), atom.name, list(atom.pos), atom.b_iso, atom.occ])
        elif res.name in AA: # amino acid residues.
            for atom in res: # Store a list of information for each atom.
                AA_chain_dict[chain.name].append([res.name, str(res.seqid), atom.name, list(atom.pos), atom.b_iso, atom.occ])

In [15]:
wat_chain_dict

{'A': [['HOH',
   '4001',
   'O',
   [-48.542, 13.195, -14.159],
   37.54999923706055,
   1.0],
  ['HOH', '4002', 'O', [-51.673, 12.791, -16.772], 61.31999969482422, 1.0],
  ['HOH', '4003', 'O', [-52.641, 7.649, -7.069], 47.20000076293945, 1.0],
  ['HOH', '4004', 'O', [-48.248, 7.953, -8.531], 28.049999237060547, 1.0],
  ['HOH', '4005', 'O', [-54.238, 9.172, -5.183], 44.189998626708984, 1.0],
  ['HOH', '4006', 'O', [-57.076, 10.675, -2.203], 46.220001220703125, 1.0],
  ['HOH', '4007', 'O', [-58.897, 16.403, 15.926], 30.93000030517578, 1.0],
  ['HOH', '4008', 'O', [-54.77, 11.026, 11.682], 22.290000915527344, 1.0],
  ['HOH', '4009', 'O', [-56.006, 10.571, 18.395], 21.43000030517578, 1.0],
  ['HOH', '4010', 'O', [-56.896, 15.207, 14.83], 41.630001068115234, 1.0],
  ['HOH', '4011', 'O', [-50.388, 3.807, 18.584], 43.970001220703125, 1.0],
  ['HOH', '4012', 'O', [-50.254, 10.001, 22.274], 21.799999237060547, 1.0],
  ['HOH', '4013', 'O', [-61.418, 23.731, 21.388], 46.91999816894531, 1.0],
  

In [16]:
AA_chain_dict

{'A': [['ASN', '8', 'N', [-54.578, 8.517, -11.22], 61.689998626708984, 1.0],
  ['ASN', '8', 'CA', [-53.242, 8.34, -11.853], 58.4900016784668, 1.0],
  ['ASN', '8', 'CB', [-53.269, 7.221, -12.888], 63.61000061035156, 1.0],
  ['ASN', '8', 'CG', [-53.732, 7.698, -14.248], 70.37000274658203, 1.0],
  ['ASN', '8', 'OD1', [-54.868, 8.146, -14.415], 74.23999786376953, 1.0],
  ['ASN', '8', 'ND2', [-52.846, 7.617, -15.229], 73.11000061035156, 1.0],
  ['ASN', '8', 'C', [-52.778, 9.646, -12.494], 54.4900016784668, 1.0],
  ['ASN', '8', 'O', [-53.583, 10.564, -12.721], 55.90999984741211, 1.0],
  ['SER', '9', 'N', [-51.47, 9.72, -12.774], 41.97999954223633, 1.0],
  ['SER', '9', 'CA', [-50.761, 10.995, -12.933], 29.1299991607666, 1.0],
  ['SER', '9', 'CB', [-51.307, 11.773, -14.137], 29.25, 1.0],
  ['SER', '9', 'OG', [-52.413, 12.593, -13.78], 29.84000015258789, 1.0],
  ['SER', '9', 'C', [-50.79, 11.886, -11.669], 21.0, 1.0],
  ['SER', '9', 'O', [-50.263, 12.978, -11.699], 18.600000381469727, 1.0],
  [

In [23]:
# For each water molecule, loop through all atoms in the AA_chain_dict and calculate the coordinates.
closest_AA_molecule={}
flat_list = [] # store just the distance values but not whole dictionary.
for chain, chain_dict in wat_chain_dict.items(): # waters.
    closest_AA={} # water data for each chain.
    for at_dict in chain_dict:
        res = at_dict[0]
        seqid = at_dict[1]
        coord = at_dict[3]
        b_fac = at_dict[4]
        occ = at_dict[5]
        closest_AA[seqid] = {'distance': None, 'AA_res': None, 'AA_seqid': None, 'wat_b_fac': None, 'wat_occ': None} 
        # store the information of closest-distance AA residue for each water.
        for chain2, chain_dict2 in AA_chain_dict.items(): # amino acid residues.
            for at_dict2 in chain_dict2:
                res2 = at_dict2[0]
                seqid2 = at_dict2[1]
                coord2 = at_dict2[3]
                # b_fac2 = at_dict2[4]
                # occ2 = at_dict2[5]
                distance = calc_distance(coord, coord2) # calculate the distance between water and any atom (of amino acid residue).
                if (closest_AA[seqid]['distance'] is None) or (closest_AA[seqid]['distance'] > distance): # update the values if it's the shortest distance.
                    closest_AA[seqid]['distance'] = distance
                    closest_AA[seqid]['AA_res'] = res2
                    closest_AA[seqid]['AA_seqid'] = seqid2
                    closest_AA[seqid]['wat_b_fac'] = b_fac # store the b-factor of water but not the b-factor of AA.
                    closest_AA[seqid]['wat_occ'] = occ # store the occupancy of water but not the occupancy of AA.
        flat_list.append([closest_AA[seqid]['distance'], closest_AA[seqid]['wat_b_fac'], closest_AA[seqid]['wat_occ']]) # each water molecule is a list.
    closest_AA_molecule[chain] = closest_AA
    


In [24]:
flat_list

[[np.float64(2.4625440097590126), 37.54999923706055, 1.0],
 [np.float64(2.8484214926867812), 61.31999969482422, 1.0],
 [np.float64(2.961266114350413), 47.20000076293945, 1.0],
 [np.float64(3.0352517193801205), 28.049999237060547, 1.0],
 [np.float64(2.943928158090818), 44.189998626708984, 1.0],
 [np.float64(2.8342515766953382), 46.220001220703125, 1.0],
 [np.float64(3.734114888430724), 30.93000030517578, 1.0],
 [np.float64(2.779171459266226), 22.290000915527344, 1.0],
 [np.float64(2.6433647118776484), 21.43000030517578, 1.0],
 [np.float64(2.613365263410379), 41.630001068115234, 1.0],
 [np.float64(2.733696398651467), 43.970001220703125, 1.0],
 [np.float64(2.6873552054017726), 21.799999237060547, 1.0],
 [np.float64(4.003644964279423), 46.91999816894531, 1.0],
 [np.float64(3.764160198503779), 48.18000030517578, 1.0],
 [np.float64(2.7023624849379497), 47.22999954223633, 1.0],
 [np.float64(4.0049450682874514), 57.880001068115234, 1.0],
 [np.float64(2.719129824042979), 44.400001525878906, 1.0

In [25]:
closest_AA_molecule

{'A': {'4001': {'distance': np.float64(2.4625440097590126),
   'AA_res': 'GLU',
   'AA_seqid': '494',
   'wat_b_fac': 37.54999923706055,
   'wat_occ': 1.0},
  '4002': {'distance': np.float64(2.8484214926867812),
   'AA_res': 'SER',
   'AA_seqid': '9',
   'wat_b_fac': 61.31999969482422,
   'wat_occ': 1.0},
  '4003': {'distance': np.float64(2.961266114350413),
   'AA_res': 'THR',
   'AA_seqid': '10',
   'wat_b_fac': 47.20000076293945,
   'wat_occ': 1.0},
  '4004': {'distance': np.float64(3.0352517193801205),
   'AA_res': 'CYS',
   'AA_seqid': '473',
   'wat_b_fac': 28.049999237060547,
   'wat_occ': 1.0},
  '4005': {'distance': np.float64(2.943928158090818),
   'AA_res': 'THR',
   'AA_seqid': '12',
   'wat_b_fac': 44.189998626708984,
   'wat_occ': 1.0},
  '4006': {'distance': np.float64(2.8342515766953382),
   'AA_res': 'THR',
   'AA_seqid': '12',
   'wat_b_fac': 46.220001220703125,
   'wat_occ': 1.0},
  '4007': {'distance': np.float64(3.734114888430724),
   'AA_res': 'VAL',
   'AA_seqid'

In [26]:
def gen_distance_data(wat_chain_dict, AA_chain_dict):
    """
    Given the dictionary of water and amino acid chains (of one PDB file), generate a flat list of distance/b-factor/occupancy values. 
    Distance is defined as the distance between water and the closest residue.
    """
    flat_list = [] # store just the distance values but not whole dictionary.
    for chain, chain_dict in wat_chain_dict.items(): # waters.
        closest_AA={} # water data for each chain.
        for at_dict in chain_dict:
            res = at_dict[0]
            seqid = at_dict[1]
            coord = at_dict[3]
            b_fac = at_dict[4]
            occ = at_dict[5]
            closest_AA[seqid] = {'distance': None, 'AA_res': None, 'AA_seqid': None, 'wat_b_fac': None, 'wat_occ': None} 
            # store the information of closest-distance AA residue for each water.
            for chain2, chain_dict2 in AA_chain_dict.items(): # amino acid residues.
                for at_dict2 in chain_dict2:
                    res2 = at_dict2[0]
                    seqid2 = at_dict2[1]
                    coord2 = at_dict2[3]
                    # b_fac2 = at_dict2[4]
                    # occ2 = at_dict2[5]
                    distance = calc_distance(coord, coord2) # calculate the distance between water and any atom (of amino acid residue).
                    if (closest_AA[seqid]['distance'] is None) or (closest_AA[seqid]['distance'] > distance): # update the values if it's the shortest distance.
                        closest_AA[seqid]['distance'] = distance
                        closest_AA[seqid]['AA_res'] = res2
                        closest_AA[seqid]['AA_seqid'] = seqid2
                        closest_AA[seqid]['wat_b_fac'] = b_fac # store the b-factor of water but not the b-factor of AA.
                        closest_AA[seqid]['wat_occ'] = occ # store the occupancy of water but not the occupancy of AA.
            flat_list.append([closest_AA[seqid]['distance'], closest_AA[seqid]['wat_b_fac'], closest_AA[seqid]['wat_occ']]) # each water molecule is a list.

    return flat_list

In [ ]:
# Training set and test set. Generate flat lists for each set.
train_distance_data, test_distance_data = {}, {}
for data, pdbs in zip([train_distance_data, test_distance_data], [pdb_train, pdb_test]):
    data={'name':[], 'distance':[], 'b_fac':[], 'occ':[]}
    for pdb in pdbs:
        name = a.stem.split('_')[0]
        structure = gemmi.read_structure(str(a))
        # Check model.
        i = 0
        model = structure[i]  # consider the first model (skip if empty)
        while len(model) == 0:  # sometimes the first model is empty
            i += 1
            try:
                model = structure[i]
            except Exception:
                raise ValueError("Can't read valid model from the input PDB file!")
            
        # Generate the dictionary of water and amino acid atoms for each PDB file.
        wat_chain_dict, AA_chain_dict={}, {}
        for chain in model:
            wat_chain_dict[chain.name] = []
            AA_chain_dict[chain.name] = []
            for res in chain:
                # Distinguish between water and amino acid.
                if res.name in water_filter: # water molecules.
                    for atom in res: # Store a list of information for each atom.
                        wat_chain_dict[chain.name].append([res.name, str(res.seqid), atom.name, list(atom.pos), atom.b_iso, atom.occ])
                elif res.name in AA: # amino acid residues.
                    for atom in res: # Store a list of information for each atom.
                        AA_chain_dict[chain.name].append([res.name, str(res.seqid), atom.name, list(atom.pos), atom.b_iso, atom.occ])
        
        # Generate the flat list of distance/b-factor/occupancy values.
        flat_list = gen_distance_data(wat_chain_dict, AA_chain_dict)
        data['name'].append(name)
        for water in flat_list:
            data['distance'].append(flat_list[0])
            data['b_fac'].append(flat_list[1])
            data['occ'].append(flat_list[2])
        

In [31]:
subprocess.run(["python", "gen_dist_bfac.py"])

KeyboardInterrupt: 

In [27]:
# # Create the dictionary of atom lists for BOTH training and test sets FOR ALL ATOMS.
# data1, data2 = pdb_train, pdb_test
# Wat_at_train, Wat_at_test = {}, {}

# # each list in 'at' contains: [res, seqid, atom, coord, b_fac, occ].
# for at_dict, data in zip([Wat_at_train, Wat_at_test], [data1, data2]):
#     for a in data: # Loop over all PDB structures.
#         name = a.stem.split('_')[0]
#         structure = gemmi.read_structure(str(a))
#         # Check model.
#         i = 0
#         model = structure[i]  # consider the first model (skip if empty)
#         while len(model) == 0:  # sometimes the first model is empty
#             i += 1
#             try:
#                 model = structure[i]
#             except Exception:
#                 raise ValueError("Can't read valid model from the input PDB file!")
            
#         # Generate the dictionary of atom lists for each PDB file.
#         chain_dict={}
#         for chain in model:
#             chain_dict[chain.name] = []
#             for res in chain:
#                 if res.name in water_filter:
#                     for atom in res: # Store a list of information for each atom.
#                         chain_dict[chain.name].append([res.name, str(res.seqid), atom.name, list(atom.pos), atom.b_iso, atom.occ]) 
#         at_dict[name] = chain_dict

*Takes too long to run codes in notebook. Run python script in HPC instead.*